<a href="https://colab.research.google.com/github/cosmic643/Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Six Major Steps of Desigining a Chatbot: 


1. Import Corpus: Corpus is the training data required for the chatbot to learn
2. Preprocessing the Data and test case handling: Converting bot input and output to either upper or lower case 
4. Tokenization: It is a structured process if converting a sentence into individual collection of words
5. Stemming: Stemming is the process of finding similarities between words with same root words i.e., reducing inflected words too their word stem
6. Bag of Words: Process of converting word in numbers by generating vector embeddings from the tokens generated i.e., is word is assigned a vector
7. One hot encoding: This is the method by which categorical data is converted to be in a machine readable form

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import nltk
import string
import random
nltk.download('all')

In [ ]:
f = open('/content/drive/MyDrive/chatbot.txt','r',errors = 'ignore')
raw_doc = f.read()
raw_doc = raw_doc.lower() #converts text to lower case
nltk.download('punkt') #Using the punkt tokenizer
nltk.download('wordnet') #using the wordnet dictionary
sent_tokens = nltk.sent_tokenize(raw_doc) #converts doc to list of sentences
word_tokens = nltk.word_tokenize(raw_doc) #converts doc to list of words

In [ ]:
sent_tokens[:2]

In [ ]:
word_tokens[:2]

Text Preprocessing

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()
#Wordnet is a semantically oriented dictionary of english  included in NLTK
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation)
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

Defining the Greeting Function

In [ ]:
greet_inputs = ("hello","hi","greetings","sup","what's up","hey")
greet_resp = ("Hi","Hello","*node*","Hi there!","Hey")
def greet(sentence):
  for word in sentence.split():
    if word.lower()in greet_inputs:
      return random.choice(greet_resp)


Response Generation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def response(user_repsonse):
  robo1_response = ''
  TfidVec = TfidfVectorizer(tokenizer = LemNormalize,stop_words='english')
  tfidf = TfidVec.fit_transform(sent_tokens)
  vals = cosine_similarity(tfidf[-1],tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if(req_tfidf == 0):
    robo1_response = robo1_response + "I am sorry! I don't understand you"
    return robo1_response
  else:
    robo1_response = robo1_response + sent_tokens[idx]
    return robo1_response


Defining conversation start/end protocol

In [ ]:
flag = True
print("BOT: My name is April. Let's have a conversation! Also, if you want to exit any time, just type Bye!")
while(flag):
  user_response = input()
  user_response = user_response.lower()
  if(user_response != 'bye'):
    if(user_response == 'thanks' or user_response == 'thank you'):
      flag = False
      print("Bot: You are welcome..")
    else:
      if(greet(user_response)!= None):
        print("BOT: "+greet(user_response))
      else:
        sent_tokens.append(user_response)
        word_tokens = word_tokens+nltk.word_tokenize(user_response)
        final_words = list(set(word_tokens))
        print("BOT: ",end="")
        print(response(user_response))
        sent_tokens.remove(user_response)
  else:
    flag = False
    print("BOT: Goodbye! Take Care")

